In [5]:
import pandas as pd

# Load the uploaded CSV file to examine its contents
data = pd.read_csv('LCD_sample_csv.csv')

# Display basic information and the first few rows of the dataset
data_info = data.info()
data_head = data.head()

data_info, data_head


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 772 entries, 0 to 771
Data columns (total 86 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   STATION                                       772 non-null    object 
 1   STATION_NAME                                  772 non-null    object 
 2   ELEVATION                                     772 non-null    float64
 3   LATITUDE                                      772 non-null    float64
 4   LONGITUDE                                     772 non-null    float64
 5   DATE                                          772 non-null    object 
 6   reportType                                    772 non-null    object 
 7   HourlySkyConditions                           742 non-null    object 
 8   HourlyVisibility                              742 non-null    float64
 9   HourlyPresentWeatherType                      107 non-null    obj

(None,
       STATION                               STATION_NAME  ELEVATION  LATITUDE  \
 0  WBAN:94846  CHICAGO OHARE INTERNATIONAL AIRPORT IL US      201.8    41.995   
 1  WBAN:94846  CHICAGO OHARE INTERNATIONAL AIRPORT IL US      201.8    41.995   
 2  WBAN:94846  CHICAGO OHARE INTERNATIONAL AIRPORT IL US      201.8    41.995   
 3  WBAN:94846  CHICAGO OHARE INTERNATIONAL AIRPORT IL US      201.8    41.995   
 4  WBAN:94846  CHICAGO OHARE INTERNATIONAL AIRPORT IL US      201.8    41.995   
 
    LONGITUDE              DATE reportType  HourlySkyConditions  \
 0   -87.9336  2014-03-01 00:51      FM-15  SCT:04 70 OVC:08 90   
 1   -87.9336  2014-03-01 01:51      FM-15            OVC:08 50   
 2   -87.9336  2014-03-01 02:51      FM-15            OVC:08 29   
 3   -87.9336  2014-03-01 03:51      FM-15              VV:09 9   
 4   -87.9336  2014-03-01 04:51      FM-15             OVC:08 8   
 
    HourlyVisibility HourlyPresentWeatherType  ...  \
 0             10.00                     

In [6]:
# Step 1: Data Cleaning
# Drop columns with excessive missing data (more than 70% missing values)

# Calculate the threshold for missing values (here set to 70%)
missing_threshold = 0.7 * len(data)

# Drop columns with missing values above the threshold
data_cleaned = data.dropna(thresh=missing_threshold, axis=1)

# Check the shape of the cleaned data to see how many columns remain
data_cleaned.shape, data_cleaned.columns.tolist()


((772, 23),
 ['STATION',
  'STATION_NAME',
  'ELEVATION',
  'LATITUDE',
  'LONGITUDE',
  'DATE',
  'reportType',
  'HourlySkyConditions',
  'HourlyVisibility',
  'HourlyDryBulbTemperatureF',
  'HourlyDryBulbTemperatureC',
  'HourlyWetBulbTemperatureF',
  'HourlyWetBulbTemperatureC',
  'HourlyDewPointTemperatureF',
  'HourlyDewPointTemperatureC',
  'HourlyRelativeHumidity',
  'HourlyWindSpeed',
  'HourlyWindDirection',
  'HourlyStationPressure',
  'HourlySeaLevelPressure',
  'HourlyAltimeterSetting',
  'DailySunrise',
  'DailySunset'])

In [7]:
# Step 2: Feature Engineering
# Convert 'DATE' column to datetime format and extract date-based features (e.g., month, day of the year)
data_cleaned['DATE'] = pd.to_datetime(data_cleaned['DATE'], errors='coerce')

# Extract month and day of the year as features
data_cleaned['Month'] = data_cleaned['DATE'].dt.month
data_cleaned['DayOfYear'] = data_cleaned['DATE'].dt.dayofyear

# Create daily averages for relevant hourly features
# Group by date to aggregate per day

# Convert relevant columns to numeric, handling errors
for col in ['HourlyDryBulbTemperatureF', 'HourlyWetBulbTemperatureF', 'HourlyDewPointTemperatureF',
            'HourlyRelativeHumidity', 'HourlyWindSpeed', 'HourlyStationPressure',
            'HourlySeaLevelPressure', 'HourlyAltimeterSetting', 'HourlyVisibility']:
    data_cleaned[col] = pd.to_numeric(data_cleaned[col], errors='coerce')

daily_features = data_cleaned.groupby('DATE').agg({
    'HourlyDryBulbTemperatureF': 'mean',
    'HourlyWetBulbTemperatureF': 'mean',
    'HourlyDewPointTemperatureF': 'mean',
    'HourlyRelativeHumidity': 'mean',
    'HourlyWindSpeed': 'mean',
    'HourlyStationPressure': 'mean',
    'HourlySeaLevelPressure': 'mean',
    'HourlyAltimeterSetting': 'mean',
    'HourlyVisibility': 'mean'
}).rename(columns=lambda x: 'DailyAvg_' + x)

# Reset index to merge back with the original data (if necessary)
daily_features.reset_index(inplace=True)

# Display the resulting daily features to check
daily_features.head()

<ipython-input-7-45bf59961258>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned['DATE'] = pd.to_datetime(data_cleaned['DATE'], errors='coerce')
<ipython-input-7-45bf59961258>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned['Month'] = data_cleaned['DATE'].dt.month
<ipython-input-7-45bf59961258>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

,DATE,DailyAvg_HourlyDryBulbTemperatureF,DailyAvg_HourlyWetBulbTemperatureF,DailyAvg_HourlyDewPointTemperatureF,DailyAvg_HourlyRelativeHumidity,DailyAvg_HourlyWindSpeed,DailyAvg_HourlyStationPressure,DailyAvg_HourlySeaLevelPressure,DailyAvg_HourlyAltimeterSetting,DailyAvg_HourlyVisibility
0,2014-03-01 00:51:00,30.0,26.0,16.0,56.0,9.0,29.28,30.02,30.00,10.00
1,2014-03-01 01:51:00,30.0,26.0,17.0,58.0,0.0,29.27,30.02,29.99,10.00
2,2014-03-01 02:51:00,29.0,26.0,21.0,72.0,8.0,29.28,30.02,30.00,6.00
3,2014-03-01 03:51:00,28.0,26.0,23.0,81.0,0.0,29.29,30.04,30.01,0.75
4,2014-03-01 04:51:00,26.0,24.0,20.0,78.0,11.0,29.32,30.06,30.04,10.00


In [8]:
import numpy as np

# Simulate a crop yield variable for demonstration purposes
# Assuming crop yield is influenced by daily average temperature, humidity, and precipitation
# Generating synthetic crop yield values based on a linear combination of some features with added noise

# We will use a basic linear formula with noise to simulate yield values (in arbitrary units)
np.random.seed(0)  # for reproducibility

# Creating the simulated yield as a function of temperature, humidity, and other factors with random noise
daily_features['SimulatedYield'] = (
    0.3 * daily_features['DailyAvg_HourlyDryBulbTemperatureF'] +
    0.5 * daily_features['DailyAvg_HourlyRelativeHumidity'] +
    np.random.normal(0, 10, len(daily_features))  # adding random noise
)

# Display the dataset with simulated crop yield
daily_features.head()


,DATE,DailyAvg_HourlyDryBulbTemperatureF,DailyAvg_HourlyWetBulbTemperatureF,DailyAvg_HourlyDewPointTemperatureF,DailyAvg_HourlyRelativeHumidity,DailyAvg_HourlyWindSpeed,DailyAvg_HourlyStationPressure,DailyAvg_HourlySeaLevelPressure,DailyAvg_HourlyAltimeterSetting,DailyAvg_HourlyVisibility,SimulatedYield
0,2014-03-01 00:51:00,30.0,26.0,16.0,56.0,9.0,29.28,30.02,30.00,10.00,54.640523
1,2014-03-01 01:51:00,30.0,26.0,17.0,58.0,0.0,29.27,30.02,29.99,10.00,42.001572
2,2014-03-01 02:51:00,29.0,26.0,21.0,72.0,8.0,29.28,30.02,30.00,6.00,54.487380
3,2014-03-01 03:51:00,28.0,26.0,23.0,81.0,0.0,29.29,30.04,30.01,0.75,71.308932
4,2014-03-01 04:51:00,26.0,24.0,20.0,78.0,11.0,29.32,30.06,30.04,10.00,65.475580


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Select features and target
X = daily_features[['DailyAvg_HourlyDryBulbTemperatureF', 'DailyAvg_HourlyRelativeHumidity']]
y = daily_features['SimulatedYield']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)




In [13]:
# Step 1: Identify and Handle Missing Values in Target Variable (y)

# Check for NaN values in y_train
print(y_train.isnull().sum())  # Check the count of NaN values

# Option 1: Remove rows with NaN in target
X_train = X_train[y_train.notna()]
y_train = y_train[y_train.notna()]

# Option 2: Impute missing values with mean or median (if appropriate)
# from sklearn.impute import SimpleImputer
# imputer = SimpleImputer(strategy='mean')  # Use 'median' if more suitable
# y_train = imputer.fit_transform(y_train.values.reshape(-1, 1))
# y_train = pd.Series(y_train.flatten(), index=X_train.index)

# Step 2: Re-train the Model

# Model training
model = RandomForestRegressor()
model.fit(X_train, y_train)

23


RandomForestRegressor()

In [16]:
# Step 1: Identify and Handle Missing Values in Target Variable (y) and X_test

# Check for NaN values in y_test and X_test
print(y_test.isnull().sum())  # Check the count of NaN values in the target
print(X_test.isnull().sum())  # Check the count of NaN values in features

# Option 1: Remove rows with NaN in target or features
X_test = X_test[y_test.notna()] #Remove rows with NaN in y_test from X_test as well
y_test = y_test[y_test.notna()]
X_test = X_test.dropna() #Remove rows with NaN in any feature in X_test
y_test = y_test[X_test.index] #Reindex y_test to match the filtered X_test


# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)

7
DailyAvg_HourlyDryBulbTemperatureF    7
DailyAvg_HourlyRelativeHumidity       7
dtype: int64
Mean Absolute Error (MAE): 10.104071956615751
Root Mean Squared Error (RMSE): 12.386624594170799
